In [2]:

import gridstatusio
client = gridstatusio.GridStatusClient(api_key = '9c97c76ced574dfdaf7b4a9618387599' )
client

GridStatusClient(host=https://api.gridstatus.io/v1)

In [3]:
df = client.get_dataset(
                        dataset= "ercot_lmp_by_bus",
                        start= "2023-07-01",
                        end="2023-07-02",    timezone="US/Central",
                    )

print(df.head())

Fetching Page 1...

SSLError: HTTPSConnectionPool(host='api.gridstatus.io', port=443): Max retries exceeded with url: /v1/datasets/ercot_lmp_by_bus/query?start_time=2023-07-01+00%3A00%3A00&end_time=2023-07-02+00%3A00%3A00&page=1&timezone=US%2FCentral&cursor=&filter_operator=%3D&return_format=json&json_schema=array-of-arrays (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))